将 Tensorflow 导入您的程序：

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from keras.layers import Dense, Flatten, Conv2D
from keras import Model

TensorFlow version: 2.10.0


加载并准备 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/)。

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

使用 `tf.data` 来将数据集切分为 batch 以及混淆数据集：

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

使用 Keras [模型子类化 API](https://tensorflow.google.cn/guide/keras/custom_layers_and_models) 构建 `tf.keras` 模型：

In [6]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

为训练选择优化器与损失函数： 

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

选择衡量指标来度量模型的损失值（loss）和准确率（accuracy）。这些指标在 epoch 上累积值，然后打印出整体结果。

In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

使用 `tf.GradientTape` 来训练模型：

In [9]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

测试模型：

In [10]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.13729450106620789, Accuracy: 95.91000366210938, Test Loss: 0.06855318695306778, Test Accuracy: 97.65999603271484
Epoch 2, Loss: 0.04416985809803009, Accuracy: 98.62333679199219, Test Loss: 0.05254289135336876, Test Accuracy: 98.20999908447266
Epoch 3, Loss: 0.02303110621869564, Accuracy: 99.24832916259766, Test Loss: 0.04772176966071129, Test Accuracy: 98.5199966430664
Epoch 4, Loss: 0.014066722244024277, Accuracy: 99.52999877929688, Test Loss: 0.05335443466901779, Test Accuracy: 98.47999572753906
Epoch 5, Loss: 0.010241428390145302, Accuracy: 99.6683349609375, Test Loss: 0.054505880922079086, Test Accuracy: 98.45999908447266


该图片分类器现在在此数据集上训练得到了接近 98% 的准确率（accuracy）。要了解更多信息，请阅读 [TensorFlow 教程](https://tensorflow.google.cn/tutorials/keras)。